In [6]:
# DataSource : http://www.cs.cornell.edu/people/pabo/movie-review-data/
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
!tar -xvf rt-polaritydata.tar.gz

--2019-02-15 11:17:16--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20, 64:ff9b::84ec:cf14
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 487770 (476K) [application/x-gzip]
Saving to: ‘rt-polaritydata.tar.gz’

100%[======================================>] 487,770     --.-K/s   in 0.008s  

2019-02-15 11:17:17 (56.8 MB/s) - ‘rt-polaritydata.tar.gz’ saved [487770/487770]

rt-polaritydata.README.1.0.txt
rt-polaritydata/rt-polarity.neg
rt-polaritydata/rt-polarity.pos


In [2]:
!/home/sunil.patel/anaconda3/bin/python -m pip install torchtext

    100% |████████████████████████████████| 71kB 3.3MB/s ta 0:00:011
You are using pip version 9.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from torchtext import data
import nltk
import json
from torchtext import vocab

import torch
import random


/home/sunil.patel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sunil.patel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sunil.patel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sunil.patel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
SEED = 1
split = 0.80

In [7]:
data_block = []
negative_data  = open('rt-polaritydata/rt-polarity.neg',encoding='utf8',errors='ignore').read().splitlines()
for i in negative_data:
        data_block.append({"sentiment":str(i.strip()),"label" : 0}) 
positve_data  = open('rt-polaritydata/rt-polarity.pos',encoding='utf8',errors='ignore').read().splitlines()
for i in positve_data:
        data_block.append({"sentiment":str(i.strip()),"label" : 1}) 

In [8]:
random.shuffle(data_block)

train_file = open('train.json', 'w')
test_file = open('test.json', 'w')
split = 0.8
for i in  range(0,int(len(data_block)*split)):
    train_file.write(str(json.dumps(data_block[i]))+"\n")
for i in  range(int(len(data_block)*split),len(data_block)):
    test_file.write(str(json.dumps(data_block[i]))+"\n")

In [9]:
def tokenize(sentiments):
#     print(sentiments)
    return sentiments

In [10]:
SENTIMENT = data.Field(preprocessing=tokenize)
LABEL = data.Field(is_target=True)
fields = {'sentiment': ('sentiment', SENTIMENT), 'label': ('label', LABEL)}

In [11]:
train_data , test_data = data.TabularDataset.splits(
                            path = '',
                            train = 'train.json',
                            test = 'test.json',
                            format = 'json',
                            fields = fields                                
)

In [12]:
print(vars(train_data[4]))

{'sentiment': ["burns'", 'fifth', 'beer-soaked', 'film', 'feels', 'in', 'almost', 'every', 'possible', 'way', '--', 'from', 'the', 'writing', 'and', 'direction', 'to', 'the', 'soggy', 'performances', '--', 'tossed', 'off', '.'], 'label': 0}


In [13]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [14]:
print('Number of training examples: ', len(train_data))
print('Number of validation examples: ', len(valid_data))
print('Number of testing examples: ',len(test_data))

Number of training examples:  5938
Number of validation examples:  2545
Number of testing examples:  2131


In [15]:
SENTIMENT.build_vocab(train_data, max_size=25000)
print("Unique tokens in SENTIMENT vocabulary:",len(SENTIMENT.vocab))

Unique tokens in SENTIMENT vocabulary: 15694


In [16]:
print(SENTIMENT.vocab.freqs.most_common(20))

[('.', 7777), ('the', 5641), (',', 5568), ('a', 4043), ('and', 3445), ('of', 3397), ('to', 2317), ('is', 1861), ('in', 1453), ('that', 1338), ('it', 1309), ('as', 1018), ('but', 942), ('with', 883), ('film', 826), ('this', 807), ('for', 760), ('an', 757), ('movie', 713), ('its', 700)]


In [17]:
print(SENTIMENT.vocab.itos[:10])

['<unk>', '<pad>', '.', 'the', ',', 'a', 'and', 'of', 'to', 'is']


In [18]:
for i in valid_data.sentiment:
    print(i)

['apuestas', 'fuertes', 'para', 'el', 'futuro', 'del', 'director', ',', 'y', 'apuestas', 'bien', 'fundadas', ',', 'pues', 'la', 'suerte', 'ya', 'la', 'tiene', ',', 'y', 'la', 'cinta', 'lo', 'comprueba', '.', '.', '.', '.']
['focuses', 'on', "joan's", 'raging', 'hormones', 'and', 'sledgehammers', 'the', 'audience', 'with', 'spanish', 'inquisitions', 'about', 'her', '"', 'madness', '"', 'so', 'much', 'that', 'i', 'became', 'mad', 'that', 'i', 'wasted', '123', 'minutes', 'and', '$9', '.', '50', 'on', 'this', '21st', 'century', 'torture', 'device', '.']
['reno', 'himself', 'can', 'take', 'credit', 'for', 'most', 'of', 'the', "movie's", 'success', '.', "he's", 'one', 'of', 'the', 'few', "'cool'", 'actors', 'who', 'never', 'seems', 'aware', 'of', 'his', 'own', 'coolness', '.']
["there's", 'back-stabbing', ',', 'inter-racial', 'desire', 'and', ',', 'most', 'importantly', ',', 'singing', 'and', 'dancing', '.']
['the', 'fact', 'that', 'the', 'rookie', 'is', 'a', 'nearly', 'impeccable', 'cinemat

In [ ]:
vec = vocab.GloVe(name = "6B")

.vector_cache/glove.6B.zip:  28%|██▊       | 244M/862M [07:12<15:54, 648kB/s]     

In [172]:
txt_field.build_vocab(trainds, valds, max_size=100000, vectors=vec)

TypeError: object of type 'int' has no len()